# <span style="color:#91299A">Основа аспирантской диссертации - проблема наблюдаемости системы</span> 

![](../storage/banners/7.jpg)

### <span style="color:#0ab49a">Согласно статье</span> <span style="color:#A254FC">038 (Shauying R.K.) Observability of Nonlinear Systems</span> 

#### <span style="color:#2c3e50">Начало</span> 

In [118]:
from sympy import *
import numpy as np

Radius_orbit=6750000.
RadiusEarth = 6371000.
mu=398576057600000.06
h_orb = Radius_orbit - RadiusEarth
ω_orb = np.sqrt(mu / (Radius_orbit ** 3))
v_orb = ω_orb * Radius_orbit

t = -131.21 + 0.00299 * h_orb
p = 2.488 * ((t + 273.1) / 216.6) ** -11.388
ρ = p / (0.2869 * (t + 273.1))

print(f"Высота орбиты: {int(h_orb // 1e3)} км\nПериод орбиты: {round((2*np.pi/ω_orb) / 3600, 2)} часов\nПлотность атмосферы: {ρ} кг/м³")

Высота орбиты: 379 км
Период орбиты: 1.53 часов
Плотность атмосферы: 1.1617593224677154e-11 кг/м³


#### <span style="color:#2c3e50">Алгоритм</span>

<span style="color:#2b817d">Примечание:</span>     $km \geq n$

In [ ]:
# Функции вспомогательные
def quat2matrix(q) -> Matrix:
    """Преобразует единичный кватернион в матрицу поворота
    :param q: Кватернион, list длинны 4
    :return: Матрица поворота, sympy.Matrix"""
    return Matrix([[1 - 2*q[2]**2 - 2*q[3]**2, 2*q[1]*q[2] + 2*q[3]*q[0], 2*q[1]*q[3] - 2*q[2]*q[0]],
                   [2*q[1]*q[2] - 2*q[3]*q[0], 1 - 2*q[1]**2 - 2*q[3]**2, 2*q[1]*q[3] + 2*q[1]*q[0]],
                   [2*q[1]*q[3] + 2*q[2]*q[0], 2*q[2]*q[3] - 2*q[1]*q[0], 1 - 2*q[1]**2 - 2*q[2]**2]])

def local_dipol(r: list, r_abs, ind: str, q: list):
    """Функция возвращает усиление антенны полуволногого диполя
    :param r: направление на аппарат в орбитальной системе координат (ОСК)
    :param r_abs: модуль вектора r (для таких лентяев как йа)
    :param ind: ось x/y/z, по которой направлена антенна в собственной системе координат (ССК)"""
    r_ = quat2matrix(q) @ Matrix(r)
    cos_a = r_[0]*int(ind=='x') + r_[1]*int(ind=='y') + r_[2]*int(ind=='z')
    sin_a = sqrt(r_[1]**2 + r_[2]**2)*int(ind=='x') + \
            sqrt(r_[0]**2 + r_[2]**2)*int(ind=='y') + \
            sqrt(r_[0]**2 + r_[1]**2)*int(ind=='z')
    return cos(cos_a * pi / 2) / sin_a / r_abs

def get_vars(name: str, n: int):
    s = ""
    for i in range(n):
        s += f"{name}_{i} "
    return var(s, real=True)

def get_func(name: str, n: int):
    return [Function(f"{name}_{i}")(t) for i in range(n)]

def get_params(n: int):
    t, ω = var("t ω", real=True)
    x = get_func(f"x", n)
    y = get_func(f"y", n)
    z = get_func(f"z", n)
    vx = get_func(f"v^x", n)
    vy = get_func(f"v^y", n)
    vz = get_func(f"v^z", n)
    wx = get_func(f"w^x", n)
    wy = get_func(f"w^y", n)
    wz = get_func(f"w^z", n)
    q0 = get_func(f"q^0", n)
    qx = get_func(f"q^x", n)
    qy = get_func(f"q^y", n)
    qz = get_func(f"q^z", n)
    return t, ω, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz

def save_reports(report_list: list, filename: str) -> None:
    common_report = report_list[0]
    for i in range(len(report_list) - 1):
        common_report += ("\n" + "-" * 100)*2 + "\n" + report_list[i + 1]
    f = open("observability_reoprt_" + filename + ".txt", "w")
    f.write(common_report)
    f.close()

def read_reports(filename: str) -> str:
    f = open("observability_reoprt_" + filename + ".txt", "r")
    common_report = f.read()
    f.close()
    return common_report

In [121]:
def ShauyingObservabilitySufficientCondition(n: int, X_: list, r: list, Y_: list, my_diff, testprint: bool = False):
    report = f"\033[1mКоличество чипсатов: {n}\033[0m\n"
    print(report)

    # Количество одномоментных измерений
    l = len(Y_)
    # Требуемое количество существующих производных функции измерения
    k = int(len(X_) // len(Y_))
    txt = f"Неизвестные: n = {len(X_)} (на каждый чипсат по {int(len(X_) // n)} параметров)\nИзвестные: l = {l}\n∃ производные порядка k = {len(X_) / len(Y_)}"
    report += txt + "\n"
    print(txt)
    
    # Матрица наблюдаемости системы
    H = Matrix([[Y_[0] for ll in range(l)] for kk in range(k)])
    H_one_line = []
    for kk in range(k):
        for ll in range(l):
            tmp = Y_[ll] if kk==0 else my_diff(H[kk - 1, ll])
            if testprint:
                print(f"_расчёт матрицы H_: k={(kk+1)}/{k}, l={(ll+1)}/{l}")
            H[kk, ll] = tmp
            H_one_line += [tmp]
    H = Matrix(H_one_line)
    txt = f"Размерность матрицы H: {shape(H)}"
    report += txt + "\n"
    print(txt)

    # Отречение от символов и знаков, воздадим числам былой языческий огонь
    s_r = lambda: np.random.uniform(-100, 100)
    s_v = lambda: np.random.uniform(-1, 1)
    s_w = lambda: np.random.uniform(-1e-4, 1e-4)
    q = np.array([s_v() for _ in range(4)])
    q /= np.linalg.norm(q)
    rand_params = [(ω, ω_orb), (pi, np.pi)]
    for i in range(n):
        rand_params += [(x[i], s_r()), (y[i], s_r()), (z[i], s_r()), 
                        (vx[i], s_v()), (vy[i], s_v()), (vz[i], s_v()), 
                        (wx[i], s_w()), (wy[i], s_w()), (wz[i], s_w()), 
                        (q0[i], q[0]), (qx[i], q[1]), (qy[i], q[2]), (qz[i], q[3])]

    # Якобиан матрицы наблюдаемости (J[1, 2]: 1 - измерение (H), 2 - состояние (X))
    J = Matrix([[Y_[0] for _ in range(len(X_))] for _ in range(k * l)])
    J_numb = np.array([[0. for _ in range(len(X_))] for _ in range(k * l)])
    for i in range(k * l):
        if testprint:
            print(f"_J_numb: расчёт строки_: {i+1} / {k * l}")
        for j in range(len(X_)):
            tmp = H[i].diff(X_[j])
            J[i, j] = tmp
            J_numb[i][j] = float(tmp.subs(rand_params))
    txt = f"Размерность матрицы J: {shape(J)}"
    report += txt + "\n"
    print(txt)
    
    # Достаточное условие
    def matrix_minor(arr: np.ndarray, i: int, j: int) -> float:
        return np.linalg.det(np.delete(np.delete(arr,i,axis=0), j, axis=1))
    txt = f"\nСледующие параметры не должны быть нулевыми:\n"
    report += txt + "\n"
    print(txt)
    d = []
    flag = True
    i_min = -1
    for i in range(len(X_)):
        tmp = matrix_minor(J_numb, i, i)
        d += [tmp if i == 0 else tmp / d[-1]]
        txt = f"Δ_{i} = {d[-1]}"
        report += txt + "\n"
        print(txt)
    
        # Чек наблюдаемости
        if flag:
            if abs(d[-1]) < 1e-7:
                i_min = i
                flag = False
        if not flag:
            break

    # Вывод
    if flag:
        txt = f"\n\033[1mВыполнено достаточное условие! Система наблюдаема\033[0m"
    else:
        txt = f"\n\033[1mНе выполнено достаточное условие. Нулевой параметр: Δ_{i_min} = {d[i_min]}\033[0m"
    report += txt + "\n"
    print(txt)

    return H, J, J_numb, d, report

# <span style="color:#00c4a0">Применение алгоритма на примерах</span>

#### <span style="color:#2c3e50">Наблюдаемость системы</span> <span style="color:#bb4bca">при кубсате строго на круговой орбите, без аэродинамики и углового движения,</span> <span style="color:#e60b9d">антенны изотропные</span>

In [ ]:
# Количество чипсатов
n = 1

# Инициализация
t, ω, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)
X_ = []
for i in range(n):
    X_ += [x[i], y[i], z[i], vx[i], vy[i], vz[i]]
r = [sqrt(x[i]**2 + y[i]**2 + z[i]**2) for i in range(n)]
R = [[r[i] if i==j else sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2) for i in range(n)] for j in range(n)]
# Вытягивание измерений в вектор
Y_ = []
Y_ += r.copy()
for i in range(n):
    for j in range(i):
        Y_ += [R[i][j]]

# Уравнения движения
def MyDiff_AeroOff_AttitudeOff_AntennaOff(expr, power: int = 1, vari: any = t):
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = []
    for i in range(n):
        subses += [(Derivative(x[i], t), vx[i])]
        subses += [(Derivative(y[i], t), vy[i])]
        subses += [(Derivative(z[i], t), vz[i])]
        subses += [(Derivative(vx[i], t), -2*ω*vz[i])]  # Нет учёта аэродинамики
        subses += [(Derivative(vy[i], t), -ω**2*y[i])]
        subses += [(Derivative(vz[i], t), 2*ω*vx[i] + 3*ω**2*z[i])]
    return anw.diff(vari).subs(subses).simplify()

H_1, J_1, J_numb_1, d_1, report_1_1 = ShauyingObservabilitySufficientCondition(n=n, X_=X_, r=r, Y_=Y_, my_diff=MyDiff_AeroOff_AttitudeOff_AntennaOff)

In [152]:
# save_reports([report_1_1, report_1_5, report_1_11], "AeroOff_AttitudeOff_AntennaOff")
print(read_reports("AeroOff_AttitudeOff_AntennaOff"))

Количество чипсатов: 1

Неизвестные: n = 6 (на каждый чипсат по 6 параметров)
Известные: l = 1
∃ производные порядка k = 6.0
Размерность матрицы H: (6, 1)
Размерность матрицы J: (6, 6)

Следующие параметры не должны быть нулевыми:

Δ_0 = 4.830413669120013e-25

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = 4.830413669120013e-25
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Количество чипсатов: 5

Неизвестные: n = 30 (на каждый чипсат по 6 параметров)
Известные: l = 15
∃ производные порядка k = 2.0
Размерность матрицы H: (30, 1)
Размерность матрицы J: (30, 30)

Следующие параметры не должны быть нулевыми:

Δ_0 = -1.8202978018171094e-82

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = -1.8202978018171094e-82
----------------------------------------------------------------------------------------------------
------

----

#### <span style="color:#2c3e50">Наблюдаемость системы</span> <span style="color:#00b0b9">при кубсате строго на круговой орбите, с аэродинамикой и угловым движением,</span> <span style="color:#e60b9d">антенны изотропные</span>

In [120]:
# Количество чипсатов
n = 5

# Коэффициент сопротивления
C = 1.17
m = 0.01  # 10 грамм
Jxx, Jyy, Jzz = (0.01, 0.01, 0.007)

# Инициализация
t, ω, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)
X_ = []
for i in range(n):
    X_ += [x[i], y[i], z[i], vx[i], vy[i], vz[i], qx[i], qy[i], qz[i], wx[i], wy[i], wz[i]]  # , q0[i]
r = [sqrt(x[i]**2 + y[i]**2 + z[i]**2) for i in range(n)]
R = [[r[i] if i==j else sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2) for i in range(n)] for j in range(n)]
# Вытягивание измерений в вектор
Y_ = []
Y_ += r.copy()
for i in range(n):
    for j in range(i):
        Y_ += [R[i][j]]

# Уравнения движения
def MyDiff_AeroOn_AttitudeOn_AntennaOff(expr, power: int = 1, vari: any = t):
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = []
    for i in range(n):
        M = np.zeros(3)
        subses += [(Derivative(x[i], t), vx[i])]
        subses += [(Derivative(y[i], t), vy[i])]
        subses += [(Derivative(z[i], t), vz[i])]
        subses += [(Derivative(vx[i], t), -2*ω*vz[i] - C*ρ/m * (v_orb + vy[i])**2 * (qx[i]*qy[i] - q0[i]*qz[i]))]
        subses += [(Derivative(vy[i], t), -ω**2*y[i])]
        subses += [(Derivative(vz[i], t), 2*ω*vx[i] + 3*ω**2*z[i])]
        subses += [(Derivative(q0[i], t), (-wx[i]*qx[i] - wy[i]*qy[i] - wz[i]*qz[i])/2)]
        subses += [(Derivative(qx[i], t), (wx[i]*q0[i] + wy[i]*qz[i] - wz[i]*qy[i])/2)]
        subses += [(Derivative(qy[i], t), (wy[i]*q0[i] + wz[i]*qx[i] - wx[i]*qz[i])/2)]
        subses += [(Derivative(qz[i], t), (wz[i]*q0[i] + wx[i]*qy[i] - wy[i]*qx[i])/2)]
        subses += [(Derivative(wx[i], t), (Jyy*wy[i]*wz[i] - Jzz*wy[i]*wz[i] + M[0]) / Jxx)]
        subses += [(Derivative(wy[i], t), (-Jxx*wx[i]*wz[i] + Jzz*wx[i]*wz[i] + M[1]) / Jyy)]
        subses += [(Derivative(wz[i], t), (Jxx*wx[i]*wy[i] - Jyy*wx[i]*wy[i] + M[2]) / Jzz)]
    return anw.diff(vari).subs(subses).simplify() # .simplify()

H_2, J_2, J_numb_2, d_2, report_2 = ShauyingObservabilitySufficientCondition(n=n, X_=X_, r=r, Y_=Y_, my_diff=MyDiff_AeroOn_AttitudeOn_AntennaOff, testprint=False)

Количество чипсатов: 5

Неизвестные: n = 60 (на каждый чипсат по 12 параметров)
Известные: l = 15
∃ производные порядка k = 4.0
_расчёт матрицы H_: k=1/4, l=1/15
_расчёт матрицы H_: k=1/4, l=2/15
_расчёт матрицы H_: k=1/4, l=3/15
_расчёт матрицы H_: k=1/4, l=4/15
_расчёт матрицы H_: k=1/4, l=5/15
_расчёт матрицы H_: k=1/4, l=6/15
_расчёт матрицы H_: k=1/4, l=7/15
_расчёт матрицы H_: k=1/4, l=8/15
_расчёт матрицы H_: k=1/4, l=9/15
_расчёт матрицы H_: k=1/4, l=10/15
_расчёт матрицы H_: k=1/4, l=11/15
_расчёт матрицы H_: k=1/4, l=12/15
_расчёт матрицы H_: k=1/4, l=13/15
_расчёт матрицы H_: k=1/4, l=14/15
_расчёт матрицы H_: k=1/4, l=15/15
_расчёт матрицы H_: k=2/4, l=1/15
_расчёт матрицы H_: k=2/4, l=2/15
_расчёт матрицы H_: k=2/4, l=3/15
_расчёт матрицы H_: k=2/4, l=4/15
_расчёт матрицы H_: k=2/4, l=5/15
_расчёт матрицы H_: k=2/4, l=6/15
_расчёт матрицы H_: k=2/4, l=7/15
_расчёт матрицы H_: k=2/4, l=8/15
_расчёт матрицы H_: k=2/4, l=9/15
_расчёт матрицы H_: k=2/4, l=10/15
_расчёт матрицы

KeyboardInterrupt: 

----

#### <span style="color:#2c3e50">Наблюдаемость системы</span> <span style="color:#00b0b9">при кубсате строго на круговой орбите,</span> <span style="color:#00b0b9">с аэродинамикой и угловым движением,</span> <span style="color:#15cbfc">антенны анизотропны</span>

In [ ]:
# Количество чипсатов
n = 9

# Разложение по антеннам
antennas = "xyz"
print(f"\033[1mУ каждого чипсата {len(antennas)} антенн(ы): {antennas}\033[0m")

# Коэффициент сопротивления
C = 1.17
m = 0.01  # 10 грамм
Jxx, Jyy, Jzz = (0.01, 0.01, 0.007)

# Инициализация
t, ω, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)

def quat2matrix(q):
    return Matrix([[1 - 2*q[2]**2 - 2*q[3]**2, 2*q[1]*q[2] + 2*q[3]*q[0], 2*q[1]*q[3] - 2*q[2]*q[0]],
                   [2*q[1]*q[2] - 2*q[3]*q[0], 1 - 2*q[1]**2 - 2*q[3]**2, 2*q[1]*q[3] + 2*q[1]*q[0]],
                   [2*q[1]*q[3] + 2*q[2]*q[0], 2*q[2]*q[3] - 2*q[1]*q[0], 1 - 2*q[1]**2 - 2*q[2]**2]])

def local_dipol(r: list, r_abs, ind: str, q: list):
    r_ = quat2matrix(q) @ Matrix(r)
    cos_a = r_[0]*int(ind=='x') + r_[1]*int(ind=='y') + r_[2]*int(ind=='z')
    sin_a = sqrt(r_[1]**2 + r_[2]**2)*int(ind=='x') + \
            sqrt(r_[0]**2 + r_[2]**2)*int(ind=='y') + \
            sqrt(r_[0]**2 + r_[1]**2)*int(ind=='z')
    return cos(cos_a * pi / 2) / sin_a / r_abs
    
X_ = []
r = []
R = []
for i in range(n):
    X_ += [x[i], y[i], z[i], vx[i], vy[i], vz[i], qx[i], qy[i], qz[i], wx[i], wy[i], wz[i]]  # , q0[i]

    # Разложение поступающего сигнала на составляющие:
    tmp = sqrt(x[i]**2 + y[i]**2 + z[i]**2)
    r += [tmp / sqrt(local_dipol([x[i], y[i], z[i]], tmp, c, [q0[i], qx[i], qy[i], qz[i]])) for c in antennas]
for j in range(n):
    for i in range(j):
        if i == j:
            tmp = sqrt(x[i]**2 + y[i]**2 + z[i]**2)
            R += [tmp / sqrt(local_dipol([x[i], y[i], z[i]], tmp, c, [q0[i], qx[i], qy[i], qz[i]])) for c in antennas]
        else:
            tmp = sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2)
            R += [tmp / sqrt(local_dipol([x[i]-x[j], y[i]-y[j], z[i]-z[j]], tmp, c, [q0[i], qx[i], qy[i], qz[i]])) \
                      / sqrt(local_dipol([x[i]-x[j], y[i]-y[j], z[i]-z[j]], tmp, c, [q0[j], qx[j], qy[j], qz[j]])) for c in antennas]
Y_ = R

# Уравнения движения
def MyDiff_AeroOn_AttitudeOn_AntennaOn(expr, power: int = 1, vari: any = t):
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = []
    for i in range(n):
        M = np.zeros(3)
        subses += [(Derivative(x[i], t), vx[i])]
        subses += [(Derivative(y[i], t), vy[i])]
        subses += [(Derivative(z[i], t), vz[i])]
        subses += [(Derivative(vx[i], t), -2*ω*vz[i] - C*ρ/m * (v_orb + vy[i])**2 * (qx[i]*qy[i] - q0[i]*qz[i]))]
        subses += [(Derivative(vy[i], t), -ω**2*y[i])]
        subses += [(Derivative(vz[i], t), 2*ω*vx[i] + 3*ω**2*z[i])]
        subses += [(Derivative(q0[i], t), (-wx[i]*qx[i] - wy[i]*qy[i] - wz[i]*qz[i])/2)]
        subses += [(Derivative(qx[i], t), (wx[i]*q0[i] + wy[i]*qz[i] - wz[i]*qy[i])/2)]
        subses += [(Derivative(qy[i], t), (wy[i]*q0[i] + wz[i]*qx[i] - wx[i]*qz[i])/2)]
        subses += [(Derivative(qz[i], t), (wz[i]*q0[i] + wx[i]*qy[i] - wy[i]*qx[i])/2)]
        subses += [(Derivative(wx[i], t), (Jyy*wy[i]*wz[i] - Jzz*wy[i]*wz[i] + M[0]) / Jxx)]
        subses += [(Derivative(wy[i], t), (-Jxx*wx[i]*wz[i] + Jzz*wx[i]*wz[i] + M[1]) / Jyy)]
        subses += [(Derivative(wz[i], t), (Jxx*wx[i]*wy[i] - Jyy*wx[i]*wy[i] + M[2]) / Jzz)]
    return anw.diff(vari).subs(subses).simplify() # .simplify()

H_3_9xy, J_3_9xy, J_numb_3_9xy, d_3_9xy, report_3_9_xyz = ShauyingObservabilitySufficientCondition(n=n, X_=X_, r=r, Y_=Y_, my_diff=MyDiff_AeroOn_AttitudeOn_AntennaOn, testprint=True)
report_3_9_xyz = f"\033[1mУ каждого чипсата {len(antennas)} антенн(ы): {antennas}\033[0m\n" + report_3_9_xyz

In [157]:
# save_reports([report_3_9_xyz, report_3_13_xy], "AeroOn_AttitudeOn_AntennaOn")
print(read_reports("AeroOn_AttitudeOn_AntennaOn"))

У каждого чипсата 3 антенн(ы): xyz
Количество чипсатов: 9

Неизвестные: n = 108 (на каждый чипсат по 12 параметров)
Известные: l = 108
∃ производные порядка k = 1.0
Размерность матрицы H: (108, 1)
Размерность матрицы J: (108, 108)

Следующие параметры не должны быть нулевыми:

Δ_0 = 0.0

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = 0.0
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
У каждого чипсата 2 антенн(ы): xy
Количество чипсатов: 13

Неизвестные: n = 156 (на каждый чипсат по 12 параметров)
Известные: l = 156
∃ производные порядка k = 1.0
Размерность матрицы H: (156, 1)
Размерность матрицы J: (156, 156)

Следующие параметры не должны быть нулевыми:

Δ_0 = 0.0

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = 0.0


### <span style="color:#0ab49a">Согласно книге</span> <span style="color:#A254FC">Афанасьев В.Н.) Математическая теория конструирования систем управления</span> 

In [114]:
A_single = Matrix([[0, 0, 0, 1, 0, 0], 
                   [0, 0, 0, 0, 1, 0],
                   [0, 0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 0, -2*w],
                   [0, -w**2, 0, 0, 0, 0],
                   [0, 0, 3*w**2, 2*w, 0, 0]])
A_ = Matrix([[A_single if i==j else zeros(6, 6) for j in range(n)] for i in range(n)])

In [ ]:
anw = Matrix.hstack(H_, 
                    A_.T @ H_, 
                    A_.T @ A_.T @ H_, 
                    A_.T @ A_.T @ A_.T @ H_, 
                    A_.T @ A_.T @ A_.T @ A_.T @ H_, 
                    A_.T @ A_.T @ A_.T @ A_.T @ A_.T @ H_)
shape(anw)

In [108]:
Radius_orbit = 6800e3
mu = 5.972e24 * 6.67408e-11
w_0 = np.sqrt(mu / Radius_orbit ** 3)

x0rand = [np.random.uniform(-100, 100) for _ in range(n)]
y0rand = [np.random.uniform(-100, 100) for _ in range(n)]
z0rand = [np.random.uniform(-100, 100) for _ in range(n)]
vx0rand = [np.random.uniform(-1, 1) for _ in range(n)]
vy0rand = [np.random.uniform(-1, 1) for _ in range(n)]
vz0rand = [np.random.uniform(-1, 1) for _ in range(n)]

In [109]:
subses = [(w, w_0)]
for i in range(n):
    subses += [(x[0][i], x0rand[i]), (y[0][i], y0rand[i]), (z[0][i], z0rand[i]), (vx[0][i], vx0rand[i]), (vy[0][i], vy0rand[i]), (vz[0][i], vz0rand[i])]
anw_numb = anw.subs(subses)
det(anw_numb)

In [110]:
anw_get = anw_numb[:,0:6]
anw_get

Matrix([
[111.920485834732, 0.371205798574056, 0.00408786854282351,                    0,                    0,                    0],
[115.690689327415, 0.945571542542173, 0.00702484508913071, -7.14198782418648e-5, -6.89198573440637e-7, -1.62727028742046e-8],
[69.3527105132341, -0.33995394940928, 0.00197638505615263, 0.000640287688750991,  2.17169867883479e-6,  1.80664255251497e-8],
[220.143810267416,  1.27720496915373,   0.010815389498481,     112.299618896271,     0.37249172485893,   0.0040985662016115],
[56.3423083929516, 0.543700710847932,  0.0128373453763167,     115.690689327415,    0.945571542542173,  0.00702484508913071],
[168.371828254168, 0.571075913837815, 0.00475079741361419,     68.8569993385628,    -0.34282990933196,   0.0019520313874017]])

In [111]:
det(anw_get)

0.0207281102680228

Линейная система с 1 челиком ненаблюдаема